### Bound-free Gaunt factor calculated following Karzas and Latter 1961.

The bound free Gaunt factor is defined as

$$ g_{bf} = \frac{\sigma_{bf}}{\sigma_{bf}^{K}} $$

where 

$\sigma_{bf} $ is the quantum-mechanical bound-free cross-section

$ \sigma_{bf}^{K} $ is the semi-classical Kramer's cross-section

For reference, see equation 40 of Karzas and Latter 1961.


In [63]:
%reset
import math 

# Constants in CGS units: http://www.astro.wisc.edu/~dolan/constants.html
R = 2.1798741e-11 # Rydber's constant [erg]
c = 2.99792458e10 # speed of light [cm s-1.]
e_charge = 4.8032068e-10 # electron charge [esu]
e_mass = 9.1093897e-28 # electron mass [g]
v = 1.

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


In [75]:
# General function for the gaunt factor
def get_gaunt_factor_bf(sigma_bf_below, sigma_bf_above, sigma_bf_kramer): 
    gaunt_factor = (sigma_bf_below+sigma_bf_above)/sigma_bf_kramer
    return gaunt_factor

# Kramer's cross section (39)
def get_kramer_sigma(n, E, Z): 
    
    eta = math.sqrt(Z*Z*R/E)
    rho = eta/n

    kramer_sigma = 2.**4./(3*math.sqrt(3.))*e_charge**2./(e_mass*c*v) *1./n *(rho**2/(1+rho**2))**2. 
    return kramer_sigma

# Quantum cross section going to l-1 (36)
def sigma_bf_below(n, l, E, Z): 
    
    eta = math.sqrt(Z*Z*R/E)
    rho = eta/n

    p1 = math.pi*e_charge**2/(e_mass*c*v)
    p2 = 2.**(4*l)/3.
    
    p3 = l**2 * math.factorial(n+l)
    
    l = int(l)
    
    # make sure we're counting for the extreme cases, when l = 0 or 1
    if l == 0: 
        return 0
    elif l ==1: 
        p3 = 1
    else:
        for i in range(1, l): 
            p3 *= (i**2 + eta**2)
    
    # the factorial of a negative number does not exist
    if n-l-1<0: 
        return 0
    else:
        p4 = math.factorial(2*l+1) *math.factorial(2*l-1) * math.factorial(n-l-1)
    
    p5 = math.exp(-4*eta*math.atan(1/rho))/(1-e_charge**(-2*math.pi*eta)) # do I have the arctan function right?
    # also is this an exponential or just the charge 

    p6 = rho**(2*l+2)/(1+rho**2)**(2*n-2)
    
    p7 = (G_l(l,-(l+1-n),eta,rho)-(1+rho**2)**(-2)*G_l(l,-(l-1-n),eta,rho))**2 # still need to define G_l
    
    all_together = p1*p2*p3/p4*p5*p6*p7
    
    return all_together

# Quantum cross section going to l+1 (37)
def sigma_bf_above(n, l, E, Z): 
    
    eta = math.sqrt(Z*Z*R/E)
    rho = eta/n
    
    p1 = math.pi*e_charge**2/(e_mass*c*v)
    
    p2 = 2.**(4*l+6)/3.
    

    p3= (l+1)**2 * math.factorial(n+l)

    l = int(l)
    n = int(n)
    for i in range(1,l+2): 
        p3 *= (i**2+eta**2)

    if n-l-1<0: 
        return 0
    else:
        p4 = (2*l+1)*math.factorial(2*l+1)*math.factorial(2*l+2)*math.factorial(n-l-1)*((l+1)**2+eta**2)**2
    
    p5 = math.exp(-4*eta*math.atan(1/rho))/(1-e_charge**(-2*math.pi*eta)) # is this exponential or electron charge 

    p6 = rho**(2*l+4*eta**2)/(1+rho**2)**(2*n)
    
    p7 = ((l+1-n)*G_l(l+1,-(l+1-n),eta,rho)+(l+1+n)/(1+rho**2)*G_l(l+1,-(l-n),eta,rho))**2 # still need to define G_l
    
    all_together = p1*p2*p3/p4*p5*p6*p7
    
    return all_together

# Bound free matrix element Gl (appendix C)
def G_l(l, m, eta, rho): 
    
    m = int(m)
    gl = 0
    for s in range(1,2*m): 
        gl += b_const(s, l, m, E, Z)*rho**s
    
    return gl 

# constant for the Gl free matrix element (Appendix c, eqn c8)
def b_const(s, l, m, eta, rho):
    
    if s==1:
        return 1
    elif s==2: 
        return 2*m*eta/l
    else:
        p1 = -1/(s*(s+2*l-1))*(4*eta*(s-1-m))*b_const(s-1, l, m, eta, rho)
        p2 = (2*m+2-s)(2*m+2*l+1-s)*b_const(s-2, l, m,  eta, rho)
        return p1+p2

In [76]:
n = 1.
l = 0
E = 1.
Z = 1.

s_kramer = get_kramer_sigma(n, E, Z)
s_below = sigma_bf_below(n, l, E, Z)
s_above = sigma_bf_above(n, l, E, Z)

print s_kramer, s_below, s_above, get_gaunt_factor_bf(s_below, s_above, s_kramer)

1.23609972438e-23 0 -3.92028229764e-08 -3.17149354565e+15
